# Exploratory Analysis on the Relationship Between Communication and High Ranked Game Outcomes

It is common knowledge within the high ranking League of Legends playerbase (top 1000 players or GM+ for the purposes of this exercise) that the "bait ping" communication option is used to express frustration at individuals, and can be interpreted as REDACTED.

### Step 1. Collecting and Storing Data

We will use the [Riot Games League of Legends API](https://developer.riotgames.com/docs/lol) to collect and store data. With a development key, we are limited to 20 requests per 1 second and 100 requests per 2 minutes. 

Goals:
- Fetch 100 most recent matches for each player
- Aggregate all match data into a single file

Limitations:
- For 200,000 potential matches, we will need about 4000 minutes, or 66 2/3 hours. Due to manually needing to refresh the API development key every 24 hours, let our approach query the API for batches of 50000 matches.

Storage:
- players.json containing all desired players
- matches.txt file with all desired match IDs
- match-data.json file containing match data

Steps:
- Fetch top 1000 player IDs
- Fetch list of 100 most recent match IDs for each player
- Aggregate all desired match IDs into a single file
- Fetch match data for each ID

In [49]:
from riotwatcher import LolWatcher, ApiError
from ratelimit import limits, sleep_and_retry
import json

api_key = 'RGAPI-9116b03a-d7f4-42ac-a71b-f9031ab55e0f'
lol_watcher = LolWatcher(api_key)

@sleep_and_retry
@limits(calls=10, period=12)
def limited_call(func, params):
    return func(*params)

@sleep_and_retry
@limits(calls=10, period=12)
def limited_call_kwargs(func, **kwargs):
    return func(**kwargs)

In [ ]:
challengers = limited_call(lol_watcher.league.challenger_by_queue, ['na1', 'RANKED_SOLO_5x5'])
grandmasters = limited_call(lol_watcher.league.grandmaster_by_queue, ['na1', 'RANKED_SOLO_5x5'])
masters = limited_call(lol_watcher.league.masters_by_queue, ['na1', 'RANKED_SOLO_5x5'])

c_IDs, gm_IDs, m_IDs = ([] for i in range(3))

for entry in challengers['entries']:
    c_IDs.append({'summonerId': entry['summonerId'], 'summonerName': entry['summonerName']})

for entry in grandmasters['entries']:
    gm_IDs.append({'summonerId': entry['summonerId'], 'summonerName': entry['summonerName']})

for entry in masters['entries']:
    m_IDs.append({'summonerId': entry['summonerId'], 'summonerName': entry['summonerName']})

# for player in c_IDs + gm_IDs:
#     player['puuid'] = limited_call(lol_watcher.summoner.by_id, ['na1', player['summonerId']])['puuid']

In [26]:
with open('players.json', 'w') as f:
    f.write(json.dumps(c_IDs + gm_IDs))
    f.close()

In [48]:
players = []
with open('players.json', 'r') as f:
    for i in f:
        players += json.loads(i)
    f.close()

In [52]:
matches = []

for player in players:
    matches += limited_call_kwargs(lol_watcher.match.matchlist_by_puuid, **{'region':'na1', 'puuid':player['puuid'], 'queue':420, 'start':0, 'count':100})
    matches += limited_call_kwargs(lol_watcher.match.matchlist_by_puuid, **{'region':'na1', 'puuid':player['puuid'], 'queue':420, 'start':100, 'count':100})

matches = [*set(matches)]
print(matches)

['NA1_4723671660', 'NA1_4724779809', 'NA1_4738561684', 'NA1_4728481124', 'NA1_4731502643', 'NA1_4740427814', 'NA1_4538482776', 'NA1_4703498984', 'NA1_4617797726', 'NA1_4686852309', 'NA1_4660075240', 'NA1_4700399121', 'NA1_4697611345', 'NA1_4740291000', 'NA1_4023357609', 'NA1_4719341066', 'NA1_4655118173', 'NA1_4708712371', 'NA1_4702209736', 'NA1_4734242497', 'NA1_4740755180', 'NA1_4734100611', 'NA1_4729746162', 'NA1_4644890196', 'NA1_4723994977', 'NA1_4718602984', 'NA1_4457550536', 'NA1_4723774998', 'NA1_4712094645', 'NA1_4723180348', 'NA1_4708879862', 'NA1_4665462273', 'NA1_4727656450', 'NA1_4669369935', 'NA1_4711810972', 'NA1_4645779474', 'NA1_4677147562', 'NA1_4736876007', 'NA1_4733700132', 'NA1_4660543921', 'NA1_4682421648', 'NA1_4679125690', 'NA1_4727701960', 'NA1_4684460824', 'NA1_4730930694', 'NA1_4687568839', 'NA1_4718105171', 'NA1_4724641158', 'NA1_4706943764', 'NA1_4673277054', 'NA1_4692909231', 'NA1_4635631218', 'NA1_4734722873', 'NA1_4611381473', 'NA1_4734676469', 'NA1_4721

In [55]:
with open('matches.txt', 'w') as f:
    f.write(','.join(matches))
    f.close()

In [60]:
import csv
with open('matches.txt', 'r') as f:
    reader = csv.reader(f)
    temp = list(reader)[0]
    f.close()
print(temp)

['NA1_4723671660', 'NA1_4724779809', 'NA1_4738561684', 'NA1_4728481124', 'NA1_4731502643', 'NA1_4740427814', 'NA1_4538482776', 'NA1_4703498984', 'NA1_4617797726', 'NA1_4686852309', 'NA1_4660075240', 'NA1_4700399121', 'NA1_4697611345', 'NA1_4740291000', 'NA1_4023357609', 'NA1_4719341066', 'NA1_4655118173', 'NA1_4708712371', 'NA1_4702209736', 'NA1_4734242497', 'NA1_4740755180', 'NA1_4734100611', 'NA1_4729746162', 'NA1_4644890196', 'NA1_4723994977', 'NA1_4718602984', 'NA1_4457550536', 'NA1_4723774998', 'NA1_4712094645', 'NA1_4723180348', 'NA1_4708879862', 'NA1_4665462273', 'NA1_4727656450', 'NA1_4669369935', 'NA1_4711810972', 'NA1_4645779474', 'NA1_4677147562', 'NA1_4736876007', 'NA1_4733700132', 'NA1_4660543921', 'NA1_4682421648', 'NA1_4679125690', 'NA1_4727701960', 'NA1_4684460824', 'NA1_4730930694', 'NA1_4687568839', 'NA1_4718105171', 'NA1_4724641158', 'NA1_4706943764', 'NA1_4673277054', 'NA1_4692909231', 'NA1_4635631218', 'NA1_4734722873', 'NA1_4611381473', 'NA1_4734676469', 'NA1_4721

In [ ]:
for match in temp:
    